<a href="https://colab.research.google.com/github/ozgurshn/Colab-notebooks/blob/master/LogisticRegressionTurkishTextClassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

import pandas as pd

import io

In [2]:
!ls

sample_data  train_tweets.csv


In [0]:
df = pd.read_csv('train_tweets.csv')

In [4]:
df.head()

,text,label
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz


In [0]:
raw_text = df['text']

y = df['label']

In [0]:
#Veri setindeki Türkçe stopwordlerin kaldırılması

stop_words = open('turkce-stop-words.txt', 'r').read().split()

import re
def pre_process(txt):
    #z = re.sub("[^a-zA-Z]",  " ",  str(txt))
     z = re.sub(r'[^\w\d\s]', ' ', str(txt))
     z = re.sub(r'\s+', ' ', z)
     #= str(txt)
     z = re.sub(r'^\s+|\s+?$', '', z.lower())
     return ' '.join(term
        for term in z.split()
        if term not in set(stop_words)
    )

In [0]:
processed = raw_text.apply(pre_process)


In [9]:
processed.head()

0    ulan wifi bağlıyım bağlıyken turkcell internet...
1    20 dk 1 gb internet 500 mb sadece kaşar turkce...
2    turkcell superonline reklamı kötü reklam görme...
3                                      turkcell pahalı
4                     turkcell kaş internetin cekmiyor
Name: text, dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y)
y = le.fit_transform(y)
y

array([2, 2, 2, ..., 2, 0, 1])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer =TfidfVectorizer(ngram_range=(1,2))

X_ngrams=vectorizer.fit_transform(raw_text)

In [0]:
#Train/Test Split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X_ngrams,y,test_size=0.2,stratify=y)

In [13]:
# Running the Classsifier

from sklearn.linear_model import LogisticRegression

clf=LogisticRegression(max_iter=2000)

clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
res= clf.predict(vectorizer.transform([('guzel')]))
res

array([1])

In [0]:
import numpy as np

def compute_mrr_at_k(items:list):
    """Compute the MRR (average RR) at cutoff k"""
    rr_total = 0
    
    for item in items:   
        rr_at_k = _reciprocal_rank(item[0],item[1])
        rr_total = rr_total + rr_at_k
        mrr = rr_total / 1/float(len(items))

    return mrr

def collect_preds(Y_test,Y_preds):
    """Collect all predictions and ground truth"""
    
    pred_gold_list=[[[Y_test[idx]],pred] for idx,pred in enumerate(Y_preds)]
    return pred_gold_list
             
def compute_accuracy(eval_items:list):
    correct=0
    total=0
    
    for item in eval_items:
        true_pred=item[0]
        machine_pred=set(item[1])
        
        for cat in true_pred:
            if cat in machine_pred:
                correct+=1
                break
    
    
    accuracy=correct/float(len(eval_items))
    return accuracy

def get_top_k_predictions(model,X_test,k):
    
    # get probabilities instead of predicted labels, since we want to collect top 3
    probs = model.predict_proba(X_test)

    # GET TOP K PREDICTIONS BY PROB - note these are just index
    best_n = np.argsort(probs, axis=1)[:,-k:]
    
    # GET CATEGORY OF PREDICTIONS
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    
    preds=[ item[::-1] for item in preds]
    
    return preds
   
def _reciprocal_rank(true_labels: list, machine_preds: list):
    """Compute the reciprocal rank at cutoff k"""
    
    # add index to list only if machine predicted label exists in true labels
    tp_pos_list = [(idx + 1) for idx, r in enumerate(machine_preds) if r in true_labels]

    rr = 0
    if len(tp_pos_list) > 0:
        # for RR we need position of first correct item
        first_pos_list = tp_pos_list[0]
        
        # rr = 1/rank
        rr = 1 / float(first_pos_list)

    return rr

In [16]:
# GET TOP K PREDICTIONS
model = clf
preds=get_top_k_predictions(model,X_test,2)
    
# GET PREDICTED VALUES AND GROUND TRUTH INTO A LIST OF LISTS
eval_items=collect_preds(y_test,preds)
    
# GET EVALUATION NUMBERS ON TEST SET -- HOW DID WE DO?
#logging.info("Starting evaluation...")
accuracy=compute_accuracy(eval_items)
mrr_at_k=compute_mrr_at_k(eval_items)
print("\nAccuracy={0}; MRR={1}".format(accuracy,mrr_at_k))



Accuracy=0.8984459703650163; MRR=0.785507770148175


In [0]:
#modeli export etme
import pickle
 
model_path="model.pkl"

pickle.dump(model,open(model_path, 'wb'))


In [0]:
#modeli yukleme
loaded_model = pickle.load(open('model.pkl', 'rb'))


In [0]:
res= loaded_model.predict(vectorizer.transform([('normal')]))
le.inverse_transform(res)

In [0]:
#1 olumlu
#2 olumsuz
#0 notr

In [0]:
res